In [6]:
import sys
import os
from pathlib import Path
import tensorflow as tf
import numpy as np

# Set ROOT path to access other directories in project
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import SnowDepth.data_loader as DL
import SnowDepth.data_splitter as DS
import SnowDepth.architecture as ARCH
import SnowDepth.optimal_features as OF

In [7]:
# Assign seed
seed = 18

# Select holdout AOI
holdout_aoi = "ID_BS"

# H5 directory and expected output files
h5_dir = ROOT / "data" / "h5_dir"
h5_path_HSIC = h5_dir / "HSIC" / "data_HSIC.h5"
h5_path_PCC  = h5_dir / "PCC"  / "data_PCC.h5"
h5_path_MI   = h5_dir / "MI"   / "data_MI.h5"

# If we have already written H5 files, we can skip a lot of steps here:
if all(p.exists() for p in [h5_path_HSIC, h5_path_PCC, h5_path_MI]):
    print("All H5 files already exist. Skipping feature selection and H5 generation.")
else:
    print("H5 files missing. Running feature selection and writing new H5 files")

    # SET IMPORTANT VARIABLES HERE:
    
    # Path to TIFF files
    data_dir = ROOT / "data" / "tif_files"

    # Number of features to select
    top_k = 10

    # Load dataframe
    df = DL.build_df(str(data_dir), drop_invalid=True, upper_threshold=3)
    dev_df = df[df["aoi_name"] != holdout_aoi].copy()
    hold_df = df[df["aoi_name"] == holdout_aoi].copy()

    # Run feature selection
    ff_algos = OF.optimal_feature_sets(dev_df, top_k=top_k, n_per_aoi=10000)

    # Write one H5 file per feature-set
    for name, feats in ff_algos.items():
        out_dir = h5_dir / name
        out_dir.mkdir(parents=True, exist_ok=True)
        DL.build_h5(
            data_dir=str(data_dir),
            out_dir=str(out_dir),
            write_mask=True,
            upper_threshold=3.0,
            selected_features=feats,
            out_name=f"data_{name}.h5",
        )
        print(f"Wrote new H5: {out_dir / f'data_{name}.h5'}")

All H5 files already exist. Skipping feature selection and H5 generation.


In [8]:
# HSIC Transformer data split
(X_train_HSIC, y_train_HSIC, m_train_HSIC), (X_val_HSIC, y_val_HSIC, m_val_HSIC) = DS.DL_split(
    h5_path=str(h5_path_HSIC),
    holdout_aoi=holdout_aoi,
    val_fraction=0.10,       
    seed=seed,
    patch_size=128,
    stride=64,
    min_valid_frac=0.80
)
print("HSIC Shapes:",
      "X_train", X_train_HSIC.shape, "y_train", y_train_HSIC.shape,
      "X_val",   X_val_HSIC.shape,   "y_val",   y_val_HSIC.shape)

(X_train_n_HSIC, X_val_n_HSIC) = ARCH.zscore_from_train(X_train_HSIC, X_val_HSIC)
y_train_f_HSIC, w_train_HSIC = ARCH.fill_nan_and_mask(y_train_HSIC)
y_val_f_HSIC,   w_val_HSIC   = ARCH.fill_nan_and_mask(y_val_HSIC)
X_train_n_HSIC = X_train_n_HSIC.astype("float32"); X_val_n_HSIC = X_val_n_HSIC.astype("float32")


# HSIC Transformer training
model_T_HSIC = ARCH.transformer_seg_model(
    input_shape=X_train_n_HSIC.shape[1:],  # (H, W, C)
    patch_size=16,                         # must divide 128
    d_model=256,
    depth=4,
    num_heads=4,
    mlp_dim=512,
    dropout=0.0
)

LR = 1e-3
model_T_HSIC.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)

# per-pixel weights must be 4D
w_train_HSIC_4d = w_train_HSIC[..., None].astype("float32")
w_val_HSIC_4d   = w_val_HSIC[..., None].astype("float32")

os.makedirs("Transformer_weights", exist_ok=True)
ckpt_T_HSIC = "Transformer_weights/transformer_best_HSIC.weights.h5"
callbacks_T_HSIC = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_T_HSIC, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5),
]

history_T_HSIC = model_T_HSIC.fit(
    X_train_n_HSIC, y_train_f_HSIC,
    sample_weight=w_train_HSIC_4d,
    validation_data=(X_val_n_HSIC, y_val_f_HSIC, w_val_HSIC_4d),
    epochs=50,
    batch_size=4,
    callbacks=callbacks_T_HSIC,
    verbose=1,
)

HSIC Shapes: X_train (289, 128, 128, 10) y_train (289, 128, 128, 1) X_val (32, 128, 128, 10) y_val (32, 128, 128, 1)

Epoch 1/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - MAE: 0.5994 - loss: 0.2682
Epoch 1: val_loss improved from inf to 0.08589, saving model to Transformer_weights/transformer_best_HSIC.weights.h5
73/73 ━━━━━━━━━━━━━━━━━━━━ 26s 181ms/step - MAE: 0.5978 - loss: 0.2669 - val_MAE: 0.3433 - val_loss: 0.0859 - learning_rate: 0.0010
Epoch 2/50
72/73 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - MAE: 0.3808 - loss: 0.1123
Epoch 2: val_loss did not improve from 0.08589
73/73 ━━━━━━━━━━━━━━━━━━━━ 12s 162ms/step - MAE: 0.3806 - loss: 0.1122 - val_MAE: 0.3633 - val_loss: 0.1035 - learning_rate: 0.0010
Epoch 3/50
72/73 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - MAE: 0.3422 - loss: 0.0974
Epoch 3: val_loss did not improve from 0.08589
73/73 ━━━━━━━━━━━━━━━━━━━━ 12s 158ms/step - MAE: 0.3425 - loss: 0.0975 - val_MAE: 0.3731 - val_loss: 0.1061 - learning_rate: 0.0010
Epoch 4/50
72/73 ━━━━━━━━━━━━━━━

In [ ]:
# PCC Transformer data split
(X_train_PCC, y_train_PCC, m_train_PCC), (X_val_PCC, y_val_PCC, m_val_PCC) = DS.DL_split(
    h5_path=str(h5_path_PCC),
    holdout_aoi=holdout_aoi,
    val_fraction=0.10,
    seed=seed,
    patch_size=128,
    stride=64,
    min_valid_frac=0.80
)
print("PCC Shapes:",
      "X_train", X_train_PCC.shape, "y_train", y_train_PCC.shape,
      "X_val",   X_val_PCC.shape,   "y_val",   y_val_PCC.shape)

(X_train_n_PCC, X_val_n_PCC) = ARCH.zscore_from_train(X_train_PCC, X_val_PCC)
y_train_f_PCC, w_train_PCC = ARCH.fill_nan_and_mask(y_train_PCC)
y_val_f_PCC,   w_val_PCC   = ARCH.fill_nan_and_mask(y_val_PCC)
X_train_n_PCC = X_train_n_PCC.astype("float32"); X_val_n_PCC = X_val_n_PCC.astype("float32")

# PCC Transformer training
model_T_PCC = ARCH.transformer_seg_model(
    input_shape=X_train_n_PCC.shape[1:],
    patch_size=16,
    d_model=256,
    depth=4,
    num_heads=4,
    mlp_dim=512,
    dropout=0.0
)

LR = 1e-3
model_T_PCC.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)

w_train_PCC_4d = w_train_PCC[..., None].astype("float32")
w_val_PCC_4d   = w_val_PCC[..., None].astype("float32")

os.makedirs("Transformer_weights", exist_ok=True)
ckpt_T_PCC = "Transformer_weights/transformer_best_PCC.weights.h5"
callbacks_T_PCC = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_T_PCC, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5),
]

history_T_PCC = model_T_PCC.fit(
    X_train_n_PCC, y_train_f_PCC,
    sample_weight=w_train_PCC_4d,
    validation_data=(X_val_n_PCC, y_val_f_PCC, w_val_PCC_4d),
    epochs=50,
    batch_size=4,
    callbacks=callbacks_T_PCC,
    verbose=1,
)


In [ ]:
# MI Transformer data split
(X_train_MI, y_train_MI, m_train_MI), (X_val_MI, y_val_MI, m_val_MI) = DS.DL_split(
    h5_path=str(h5_path_MI),
    holdout_aoi=holdout_aoi,
    val_fraction=0.10,
    seed=seed,
    patch_size=128,
    stride=64,
    min_valid_frac=0.80
)
print("MI Shapes:",
      "X_train", X_train_MI.shape, "y_train", y_train_MI.shape,
      "X_val",   X_val_MI.shape,   "y_val",   y_val_MI.shape)

(X_train_n_MI, X_val_n_MI) = ARCH.zscore_from_train(X_train_MI, X_val_MI)
y_train_f_MI, w_train_MI = ARCH.fill_nan_and_mask(y_train_MI)
y_val_f_MI,   w_val_MI   = ARCH.fill_nan_and_mask(y_val_MI)
X_train_n_MI = X_train_n_MI.astype("float32"); X_val_n_MI = X_val_n_MI.astype("float32")

# MI Transformer training
model_T_MI = ARCH.transformer_seg_model(
    input_shape=X_train_n_MI.shape[1:],
    patch_size=16,
    d_model=256,
    depth=4,
    num_heads=4,
    mlp_dim=512,
    dropout=0.0
)

LR = 1e-3
model_T_MI.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)

w_train_MI_4d = w_train_MI[..., None].astype("float32")
w_val_MI_4d   = w_val_MI[..., None].astype("float32")

os.makedirs("Transformer_weights", exist_ok=True)
ckpt_T_MI = "Transformer_weights/transformer_best_MI.weights.h5"
callbacks_T_MI = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_T_MI, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5),
]

history_T_MI = model_T_MI.fit(
    X_train_n_MI, y_train_f_MI,
    sample_weight=w_train_MI_4d,
    validation_data=(X_val_n_MI, y_val_f_MI, w_val_MI_4d),
    epochs=50,
    batch_size=4,
    callbacks=callbacks_T_MI,
    verbose=1,
)
